In [1]:
from IPython.display import IFrame

IFrame('https://data.fortworthtexas.gov/resource/k6ic-7kp7.json?case_no=180003105', width = 700, height = 450)


In [2]:
# This is the API endpoint - the source of our data
endpoint = "https://data.fortworthtexas.gov/resource/k6ic-7kp7.json"

# Here is the first part of the SoQL query - denoting what type of offense we want
type_query = "?$where=nature_of_call='THEFT'"

# Here is the second part of the query - we are taking offenses after midnight on December 1, 
# and prior to midnight on January 1
date_query = " AND reported_date>'2019-01-01T00:00:00' AND reported_date<'2020-01-01T00:00:00' &$limit=50000"

# We then concatenate together the endpoint and the two components of the SoQL query
# to get our API call.
theft_call = endpoint + type_query + date_query

In [3]:
IFrame(theft_call,width = 500, height = 700)

In [4]:
theft_call

"https://data.fortworthtexas.gov/resource/k6ic-7kp7.json?$where=nature_of_call='THEFT' AND reported_date>'2019-01-01T00:00:00' AND reported_date<'2020-01-01T00:00:00' &$limit=50000"

In [6]:
from urllib.parse import quote

formatted_call = quote(theft_call , safe = "/:?$=<->&")

In [7]:
formatted_call

'https://data.fortworthtexas.gov/resource/k6ic-7kp7.json?$where=nature_of_call=%27THEFT%27%20AND%20reported_date>%272019-01-01T00:00:00%27%20AND%20reported_date<%272020-01-01T00:00:00%27%20&$limit=50000'

In [9]:
import pandas as pd
data = pd.read_json(formatted_call)

In [22]:
import requests
from requests.packages.urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter


session = requests.Session()
adapter= HTTPAdapter(max_retries=3)

session.mount('https://', adapter)
session.mount('http://', adapter)

In [27]:
headers = {
    # 'Authorization' : f'Token {api_key}',
    'Contents' : 'application/json'
}

In [29]:
response = session.get(formatted_call,headers = headers)

In [36]:
data = response.json()

In [40]:
data = pd.DataFrame(data) 

In [46]:
extracted_data = data.join(pd.json_normalize(data['location_1']))

In [12]:
import http.client
conn = http.client.HTTPConnection('data.fortworthtexas.gov/resource/k6ic-7kp7.json')

In [14]:
conn.request("GET", formatted_call) 
res = conn.getresponse

gaierror: [Errno 8] nodename nor servname provided, or not known

In [122]:
import pandas as pd

theft_df = pd.read_json(formatted_call)

theft_df.shape

(12296, 17)

In [123]:
theft_df = theft_df.head(10)

In [124]:
theft_df.location_1.fillna('', inplace = True)

In [128]:
theft_df.location_1[8]

''

In [125]:
loc_1 = theft_df.join(pd.json_normalize(theft_df['location_1']))

AttributeError: 'str' object has no attribute 'items'

In [80]:
data_test = extracted_data[extracted_data['location_1'].apply(lambda x: isinstance(x,dict))]

In [82]:
data_test['human_address'].apply(lambda x: explode_columns(x))

0        [, , , ]
1        [, , , ]
2        [, , , ]
3        [, , , ]
4        [, , , ]
           ...   
12291    [, , , ]
12292    [, , , ]
12293    [, , , ]
12294    [, , , ]
12295    [, , , ]
Name: human_address, Length: 12027, dtype: object

In [68]:
def explode_columns(json_string):
    data = json.loads(json_string)
    return [data[key] for key in data]

# Apply the function to each row in the DataFrame
df_1 = pd.concat([df_1, df_1['human_address'].apply(lambda x: pd.Series(explode_columns(x)))],axis=1)


In [109]:
df_1

,case_no_offense,case_no,reported_date,nature_of_call,from_date,offense,offense_desc,block_address,city,state,...,location_type,locationtypedescription,location_1,latitude,longitude,human_address,0,1,2,3
0,190060995-26A,190060995,2019-07-16T10:02:00.000,THEFT,2019-03-16T12:00:00.000,26A,PC 31.04(E)(4) 2015 Theft of Service>=$2500<$...,6700 CAMP BOWIE BLVD,FORT WORTH,TX,...,8.0,08 DEPARTMENT/DISCOUNT STORE,"{'latitude': '32.721920974474735', 'longitude'...",32.721920974474735,-97.43228626354923,"{""address"": """", ""city"": """", ""state"": """", ""zip""...",,,,


In [113]:
column_mapping = {0:'address',1:'city',2:'state',3:'zip'}

In [119]:
df_1.rename(columns = column_mapping,inplace=True)

In [120]:
df_1.columns

Index(['case_no_offense', 'case_no', 'reported_date', 'nature_of_call',
       'from_date', 'offense', 'offense_desc', 'block_address', 'city',
       'state', 'beat', 'division', 'councildistrict', 'attempt_complete',
       'location_type', 'locationtypedescription', 'location_1', 'latitude',
       'longitude', 'human_address', 'address', 'city', 'state', 'zip'],
      dtype='object')

In [121]:
df_1

,case_no_offense,case_no,reported_date,nature_of_call,from_date,offense,offense_desc,block_address,city,state,...,location_type,locationtypedescription,location_1,latitude,longitude,human_address,address,city,state,zip
address,190060995-26A,190060995,2019-07-16T10:02:00.000,THEFT,2019-03-16T12:00:00.000,26A,PC 31.04(E)(4) 2015 Theft of Service>=$2500<$...,6700 CAMP BOWIE BLVD,FORT WORTH,TX,...,8.0,08 DEPARTMENT/DISCOUNT STORE,"{'latitude': '32.721920974474735', 'longitude'...",32.721920974474735,-97.43228626354923,"{""address"": """", ""city"": """", ""state"": """", ""zip""...",,,,


In [206]:
df_1.columns = ['address', 'city', 'state', 'zip']


ValueError: Length mismatch: Expected axis has 7 elements, new values have 4 elements

In [205]:
df

,address,city,state,zip
0,123 Main St,Anytown,CA,12345
1,456 Elm St,Othertown,NY,67890


In [62]:
extracted_data['human_address'].apply(explode_columns)


TypeError: the JSON object must be str, bytes or bytearray, not float

In [46]:
df_1['human_address'].apply(lambda x: pd.Series(explode_columns(x)))

TypeError: the JSON object must be str, bytes or bytearray, not dict

In [49]:
eval(loc_1.human_address)


TypeError: eval() arg 1 must be a string, bytes or code object

In [44]:
sample_type = type(theft_df["location_1"][0])
print(f"The data type before using apply function: {sample_type}\n")

The data type before using apply function: <class 'dict'>



In [45]:
sample_type = type(theft_df["location_1"][0])
print(f"The data type before using apply function: {sample_type}\n")

The data type before using apply function: <class 'dict'>

